In [1]:
#import dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from datetime import datetime

#import Socrata to pull API data...may need to run pip install sodapy
from sodapy import Socrata

#import password for mySQL from config.py.
from config import password

#import app_token (if desired) from app_token.py to prevent limited pull requests
# from app_token import app_token

In [2]:
#call Socrata without app_token
client = Socrata("data.cdc.gov", None)

# # Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cdc.gov", app_token)

#requests result, limit is 25000 to ensure full data is pulled
results = client.get("a3uk-kgrx", limit="25000")

# Convert to pandas DataFrame
drug_df = pd.DataFrame.from_records(results)

#create month and year string from month and year columns
drug_df["month-year"] = drug_df["month"].apply(lambda x: x[:3]) + " " + drug_df["year"]

#converty month-year column to datetime
drug_df["month-year"] = pd.to_datetime(drug_df["month-year"])

#drop duplicate entries from dataframe
drug_df = drug_df.drop_duplicates(subset=['footnote', 'indicator', 'state', 'month-year', "data_value"])
drug_df = drug_df.reset_index(drop=True)

In [3]:
#push dataframe contents to mysql

import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

dbuser = 'root'
#read in password from config.py file
dbpassword = password
dbhost = 'localhost'
dbport = '3306'

#database name is Project2 (need to create this locally)
dbname= 'project2'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")

# state_cons_df
drug_df.to_sql('drug', engine, index=True, if_exists="replace")

In [4]:
state = "NY"
indicators = drug_df['indicator'].unique()

drug_df[(drug_df['indicator']==indicators[1]) & (drug_df['state']==state)].sort_values(by='month-year')

,data_value,footnote,indicator,month,percent_complete,percent_pending_investigation,predicted_value,state,state_name,year,month-year
4616,96525,Numbers may differ from published reports usin...,Number of Deaths,January,100,0.278684278684279,NaN,NY,NY,2015,2015-01-01
4590,97079,Numbers may differ from published reports usin...,Number of Deaths,February,100,0.29460542444813,NaN,NY,NY,2015,2015-02-01
3226,97885,Numbers may differ from published reports usin...,Number of Deaths,March,100,0.306482096337539,NaN,NY,NY,2015,2015-03-01
4875,98250,Numbers may differ from published reports usin...,Number of Deaths,April,100,0.307379134860051,NaN,NY,NY,2015,2015-04-01
7054,98620,Numbers may differ from published reports usin...,Number of Deaths,May,100,0.313323869397688,NaN,NY,NY,2015,2015-05-01
1239,98695,Numbers may differ from published reports usin...,Number of Deaths,June,100,0.309032879071888,NaN,NY,NY,2015,2015-06-01
1599,98964,Numbers may differ from published reports usin...,Number of Deaths,July,100,0.322339436562791,NaN,NY,NY,2015,2015-07-01
1694,99184,Numbers may differ from published reports usin...,Number of Deaths,August,100,0.337756089691886,NaN,NY,NY,2015,2015-08-01
5886,99349,Numbers may differ from published reports usin...,Number of Deaths,September,100,0.353299982888605,NaN,NY,NY,2015,2015-09-01
4997,99526,Numbers may differ from published reports usin...,Number of Deaths,October,100,0.370757390028736,NaN,NY,NY,2015,2015-10-01


In [5]:
indicators

array(['Natural & semi-synthetic opioids (T40.2)', 'Number of Deaths',
       'Natural, semi-synthetic, & synthetic opioids, incl. methadone (T40.2-T40.4)',
       'Percent with drugs specified', 'Opioids (T40.0-T40.4,T40.6)',
       'Psychostimulants with abuse potential (T43.6)', 'Heroin (T40.1)',
       'Methadone (T40.3)',
       'Natural & semi-synthetic opioids, incl. methadone (T40.2, T40.3)',
       'Number of Drug Overdose Deaths',
       'Synthetic opioids, excl. methadone (T40.4)', 'Cocaine (T40.5)'],
      dtype=object)